In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('./mnist/data/', one_hot = True)

In [42]:
tf.reset_default_graph()

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [8]:
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])
with tf.device('/cpu:0'):
    W1 = tf.Variable(tf.random_normal([784, 256], stddev=0.01))
    b1 = tf.Variable(tf.random_normal([256]))
    L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

    W2 = tf.Variable(tf.random_normal([256, 256], stddev=0.01))
    b2 = tf.Variable(tf.random_normal([256]))
    L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

    W3 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
    b3 = tf.Variable(tf.random_normal([10]))
    L3 = tf.matmul(L2, W3) + b3

    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = Y, logits = L3))
    train = tf.train.AdamOptimizer(1e-3).minimize(cost)

prediction = tf.argmax(tf.nn.softmax(L3), 1)
labels = tf.argmax(Y, 1)
accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, labels), tf.float32))

batch_size = 100

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    total_batch = int(mnist.train.num_examples / batch_size)
    for epoch in range(15):
        total_cost = 0
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            
            _, c = sess.run([train, cost], feed_dict = {X: batch_xs, Y: batch_ys})
            total_cost += c
        acc = sess.run([accuracy], feed_dict = {X: mnist.test.images, Y: mnist.test.labels})
        print('epoch: {}, cost: {}, accuracy: {}'.format(epoch, total_cost/total_batch, acc))

epoch: 0, cost: 0.565647977753119, accuracy: [0.9327001]
epoch: 1, cost: 0.18339841230349108, accuracy: [0.95580018]
epoch: 2, cost: 0.1215089893645861, accuracy: [0.96630013]
epoch: 3, cost: 0.0889964967864481, accuracy: [0.97290015]
epoch: 4, cost: 0.06859741344370625, accuracy: [0.97370011]
epoch: 5, cost: 0.05450161150402644, accuracy: [0.9703002]
epoch: 6, cost: 0.04262884056788276, accuracy: [0.97540021]
epoch: 7, cost: 0.03474905598823997, accuracy: [0.97510016]
epoch: 8, cost: 0.028327038006179712, accuracy: [0.97630012]
epoch: 9, cost: 0.023062253095400095, accuracy: [0.97700012]
epoch: 10, cost: 0.018911648366960104, accuracy: [0.97820008]
epoch: 11, cost: 0.01572975490443324, accuracy: [0.97710013]
epoch: 12, cost: 0.013795616331732494, accuracy: [0.97510016]
epoch: 13, cost: 0.011182634777452965, accuracy: [0.97640014]
epoch: 14, cost: 0.013350546640285756, accuracy: [0.97580016]


In [11]:
tf.reset_default_graph()

In [12]:
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

def build_net_batch_norm(x, activation, is_training, reuse = None):
   # L1 = tf.layers.batch_normalization(x, training = is_training, reuse = reuse)
    L1 = tf.layers.dense(x, 256, reuse = reuse, name = 'L1_batch')
    L1 = tf.layers.batch_normalization(L1, training = is_training)
    L1 = activation(L1)
    L1 = tf.layers.dropout(L1, 0.3, training=is_training)
    
    L2 = tf.layers.dense(L1, 256,  reuse = reuse, name = 'L2_batch')
    L2 = tf.layers.batch_normalization(L2, training = is_training)
    L2 = activation(L2)
    L2 = tf.layers.dropout(L2, 0.3, training=is_training)
    
    output = tf.layers.dense(L2, 10, reuse = reuse, name = 'L3_batch')
    return output   

In [88]:
def build_net(x, activation, is_training, reuse = None):
   # L1 = tf.layers.batch_normalization(x, training = is_training, reuse = reuse)
    L1 = tf.layers.dense(x, 256, reuse = reuse, name = 'L1')
    L1 = activation(L1)
    
    L2 = tf.layers.dense(L1, 256,  reuse = reuse, name = 'L2')
    L2 = activation(L2)
    
    output = tf.layers.dense(L2, 10, reuse = reuse, name = 'L3')
    return output   

In [13]:
with tf.device('/cpu:0'):
    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    logits_batch = build_net_batch_norm(X, tf.nn.relu, True)
    output_batch = build_net_batch_norm(X, tf.nn.relu, False, reuse = True)
    cost_batch = tf.reduce_mean(tf.losses.softmax_cross_entropy(Y, logits_batch))

#with tf.control_dependencies(extra_update_ops):
train_batch = tf.train.AdamOptimizer(0.01).minimize(cost_batch)



accuracy_batch = tf.metrics.accuracy(tf.argmax(Y, 1), tf.argmax(tf.nn.softmax(output_batch), 1))

In [3]:
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
logits = build_net(X, tf.nn.relu, True)
output = build_net(X, tf.nn.relu, False, reuse = True)
cost = tf.reduce_mean(tf.losses.softmax_cross_entropy(Y, logits))

with tf.control_dependencies(extra_update_ops):
    train = tf.train.AdamOptimizer(0.01).minimize(cost)



accuracy = tf.metrics.accuracy(tf.argmax(Y, 1), tf.argmax(tf.nn.softmax(output), 1))

NameError: name 'build_net' is not defined

In [91]:
batch_size = 100

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    total_batch = int(mnist.train.num_examples / batch_size)
    for epoch in range(30):
        total_cost = 0
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            
            _, c = sess.run([train, cost], feed_dict = {X: batch_xs, Y: batch_ys})
            total_cost += c
        acc = sess.run([accuracy], feed_dict = {X: mnist.test.images, Y: mnist.test.labels})
        print('epoch: {}, cost: {}, accuracy: {}'.format(epoch, total_cost/total_batch, acc))

epoch: 0, cost: 0.24070071481845595, accuracy: [(0.0, 0.94919997)]
epoch: 1, cost: 0.1387631998041814, accuracy: [(0.94919997, 0.95270002)]
epoch: 2, cost: 0.12201243366792121, accuracy: [(0.95270002, 0.95499998)]
epoch: 3, cost: 0.10665283208374272, accuracy: [(0.95499998, 0.95740002)]
epoch: 4, cost: 0.09982649514155294, accuracy: [(0.95740002, 0.95986003)]
epoch: 5, cost: 0.09065829710484567, accuracy: [(0.95986003, 0.96108335)]
epoch: 6, cost: 0.08295000379672274, accuracy: [(0.96108335, 0.96192855)]
epoch: 7, cost: 0.07838856377003883, accuracy: [(0.96192855, 0.96249998)]
epoch: 8, cost: 0.0746744273330974, accuracy: [(0.96249998, 0.96338886)]
epoch: 9, cost: 0.0840728684372946, accuracy: [(0.96338886, 0.96379)]
epoch: 10, cost: 0.06972372749418189, accuracy: [(0.96379, 0.9647091)]
epoch: 11, cost: 0.06685490308117799, accuracy: [(0.9647091, 0.96524167)]
epoch: 12, cost: 0.0613623296494056, accuracy: [(0.96524167, 0.96523845)]
epoch: 13, cost: 0.05700320681752187, accuracy: [(0.96

In [14]:
batch_size = 100

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    total_batch = int(mnist.train.num_examples / batch_size)
    for epoch in range(30):
        total_cost = 0
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            
            _, c = sess.run([train_batch, cost_batch], feed_dict = {X: batch_xs, Y: batch_ys})
            total_cost += c
        acc = sess.run([accuracy_batch], feed_dict = {X: mnist.test.images, Y: mnist.test.labels})
        print('epoch: {}, cost: {}, accuracy: {}'.format(epoch, total_cost/total_batch, acc))

InternalError: Blas GEMM launch failed : a.shape=(100, 10), b.shape=(256, 10), m=100, n=256, k=10
	 [[Node: gradients/L3_batch/MatMul_grad/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=true, _device="/job:localhost/replica:0/task:0/gpu:0"](gradients/L3_batch/BiasAdd_grad/tuple/control_dependency, L3_batch/kernel/read/_89)]]
	 [[Node: gradients/batch_normalization/Reshape_1_grad/Reshape/_145 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_1055_gradients/batch_normalization/Reshape_1_grad/Reshape", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'gradients/L3_batch/MatMul_grad/MatMul', defined at:
  File "C:\Users\POWER USER\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\POWER USER\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_g